# Terrorist Classifier

The goal of this classifier is to predict the terrorist group responsible for an attack based on information about the incident, such as the weapons used and groups targeted.

## Data Source

Terrorist attack data from 1970-2017 were collected from the Global Terrorism Database (GTD), available for download here: https://www.start.umd.edu/gtd/contact/.

The Excel spreadsheet was converted to a .csv file to expedite handling with pandas, using a Python package:

In [ ]:
pip install xlsx2csv
xlsx2csv ./globalterrorismdb_0617dist.xlsx ./globalterrorismdb_0617dist.csv

In [ ]:
import os
os.chdir('/Users/richard/Documents/Fellowship.ai/')
import pandas as pd
df_full = pd.read_csv('GTD_0617dist/globalterrorismdb_0617dist.csv')

## Model Selection

We initially choose to use a neural network to model this supervised classification problem, as it handles irrelevant/noisy features well. Our database includes 50+ predictor variables, and we have no prior information about their statistical power. Neural networks also automatically detect feature interactions, useful as many of our variables are subsets of each other (i.e. weapon subtype 1 and weapon 1). Theoretically, we can use the hidden layer outputs of our NN to guide our feature selection for other algorithms.

We assume the data is not in a time series; i.e. each attack is independent from the subsequent or preceeding attack in time.

## Examine Response Variable

We want to first filter our data to remove incidents where the attacking group is unknown. We cannot meaningfully train or validate our model using samples with an unknown response variable. Then, we want to examine the distribution of attacks by group, to determine a threshold for inclusion in the model. Groups with just a handful of documented attacks will not provide the model with enough information to identify to recognize another instance of that group's attack.

In [63]:
df = df_full[df_full['gname'] != 'Unknown']
print("Of the {:,} documented attacks, {:,} have a known group responsible ({:.1%})".format((len(df_full)), len(df), len(df)/len(df_full)))

groups = sorted(list(set(df['gname'])))
print("There are a total of {:,} identified groups\n".format(len(groups)))
groups = {g: 0 for g in groups}
for entry in df['gname']:
    groups[entry] += 1
    
dist = pd.DataFrame()
dist['attack count'] = [1, 5, 10, 50, 100, 200, 500, 1000, 5000]
g_subset = dist['attack count'].map(lambda x: {g:c for g,c in groups.items() if c <= x})
dist['% of groups'] = g_subset.map(lambda x: len(x)) / len(groups)
dist['% of attacks'] = g_subset.map(lambda x: sum(x.values())) / len(df)
pd.options.display.float_format = '{:.1%}'.format
print(dist)

Of the 170,350 documented attacks, 92,044 have a known group responsible (54.0%)
There are a total of 3,453 identified groups

   attack count  % of groups  % of attacks
0             1        49.0%          1.8%
1             5        77.1%          4.9%
2            10        84.8%          7.1%
3            50        94.3%         15.4%
4           100        96.6%         21.6%
5           200        97.9%         28.8%
6           500        99.1%         42.5%
7          1000        99.5%         52.3%
8          5000       100.0%         92.9%


As shown above, single offenders account for almost half of the unique groups, but represent less than 2% of all documented attacks. On the other hand, groups with more than 1000 attacks represent almost half of all documented attacks, despite accounting for only 0.5% of the total groups. This indicates that a large proportion of the attacks are carried out by a few active groups.

We tentatively choose to only include groups with at least 5 documented attacks. Including more groups in the training set will allow the model to learn and recognize a larger subset of groups in the test set, but with the tradeoff of being more computationally expensive.

## Split training/test sets

Before we perform any preprocessing, we first split the data into training and test sets. This way, any data transformations can be first learnt from the training set and then applied to the held-out test set. We want to avoid using any knowledge gleaned from the test set to inform our processing of the training set. We use sklearn's train_test_split to randomly select 25% of samples for the test set.

In [ ]:
from sklearn.model_selection import train_test_split
dfy = df['gname']
dfx = df.drop('gname', axis=1)
train_x, test_x, train_y, test_y = train_test_split(dfx, dfy)

## Preprocessing

Now we can filter our training set for groups with at least 5 documented attacks.

In [47]:
group_thres = 5
newgroups = []
for group, count in groups.items():
    if count >= group_thres:
        newgroups.append(group)    
train_x = train_x[train_y.isin(newgroups)]
train_y = train_y[train_y.isin(newgroups)]

### Feature Selection

The database contains 135 variables, but many of these were excluded for the following reasons:
* Several variables are encoded as both text and unique integer codes (i.e. country = 4 corresponds to country_txt = Albania). We only include the integer encodings.
* Some information is too specific to be generalized by the model, i.e. Names of Targets, Latitude/Longitude of attack.
* Text entries (i.e. Summary, Add'l Notes) were not included as they would need to be parsed by word count.

After manually filtering for relevant features, we are left with 52 predictor variables.

In [59]:
cols = pd.read_csv('/Users/richard/Documents/Fellowship.ai/variables LONGER.txt')
cols = list(cols['headers'])
df = df[cols]

### One-Hot Encoding

45 of the 52 selected variables are categorical. Since these variables have no inherent logical ordering, we need to binarize them using one-hot encoding. Several categorical variables contain a large number of values (i.e. 190 countries), many of which are rare and thus do not carry much statistical power. Thus, we provide a method to aggregate these rare values into an "other" category, to reduce the number of dummy variables. This also identifies unknown values, which can show up as a negative integer (i.e. –9) or as NaN.

In [28]:
import numbers
def aggregate(df_or, col, threshold=1):
    df = df_or[df_or[col].notnull()]
    set_vals = list(set(df[col]))
    # for variables with fewer values, faster to slice dataframe
    if (len(set_vals) < 30):
        vals = {v: sum(df[col] == v) for v in set_vals}
    # for variables with many values, faster to iterate through elements
    else:  
        vals = {v:0 for v in set_vals}
        for index, row in df.iterrows():
            vals[row[col]] += 1
    
    used, unused, unknown = {}, {}, {}
    if len(df_or) - len(df) > 0:
        unknown['NaN'] = len(df_or) - len(df)
    for k,v in vals.items():
        if isinstance(k, numbers.Real) and k < 0:
            unknown[k] = v
        else:
            if v < threshold:
                unused[k] = v
            else:
                used[k] = v
    print(sum(unused.values())/len(df_or))
    return used, unused, unknown

These sets of common, rare, and unknown values are passed into the one-hot encoder to convert the categorical variable into a set of dummy variables:

In [21]:
def one_hot(df, col, used, unused, unknown):
    for val in used:
        df[col + '_' + str(val)] = (df[col] == val)
    if unused:
        df[col + '_other'] = df[col].isin(unused)
    if unknown:
        df[col + '_unknown'] = (df[col].isin(unknown) | df[col].isnull())
    
    # drop last column since only need k-1 dummy variables
    df.drop([col, df.columns[-1]], axis=1, inplace=True)
    return df

In [ ]:
scalars = ['nkill', 'nwound', 'nhostkid', 'nhours', 'ndays', 'ransomamt', 'nperps']
categories = [c for c in cols if c not in scalars]
cate_thres = 100
for col in categories[1:]:
    used, unused, unknown = aggregate(train_x, col, cate_thres)
    one_hot(train_x, col, used, unused, unknown)
    one_hot(test_x, col, used, unused, unknown)

The number of dummy variables can be controlled by increasing the frequency threshold for a value to be aggregated. For our first pass, we will aggregate all values with fewer than 100 occurrences. This threshold is a model hyperparameter that we will optimize later.

| Frequency Threshold | # Dummy Variables |
| --- | ---|
| 1 | 1230 |
| 10 | 727 |
| 100 | 408 |

### Cleaning scalar variables 

For scalar variables, we need to add a binary variable to designate unknown values. Certain variables have unique flag values to indicate that the value is unknown.

In [49]:
def clean_scalar(df, col, special_unknowns=None):
    unknown_rows = (df[col].isnull()) | (df[col] < 0) | \
                 (col in special_unknowns and df[col] == special_unknowns[col])
    if any(unknown_rows):        
        df[col + '_unknown'] = unknown_rows
        df.loc[unknown_rows, col] = 0
    return df

In [50]:
special_unknowns = {'nhours': 999}
for col in scalars:
    train_x = clean_scalar(train_x, col, special_unknowns)
    test_x = clean_scalar(test_x, col, special_unknowns)

### Scaling Data

We need to normalize our data since the MLP model is sensitive to feature scaling.

In [ ]:
from sklearn.preprocessing import StandardScaler
scalers = StandardScaler()
scalers.fit(train_x)
trainx = scalers.transform(train_x)
testx = scalers.transform(test_x)

## Training Model

We train the MLP model using the default number of hidden layers, and then predict the groups responsible in the test dataset.

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(20,20))
model.fit(trainx, train_y)
pred = model.predict(testx)

## Evaluating Performance

We use sklearn's classification report to evaluate model performance over different data filtering thresholds. As mentioned above, we are currently including groups with at least 5 documented attacks, and aggregating feature values with fewer than 100 instances. Below we evaluate how model performance and efficiency change with these two hyperparameters.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y,pred))

| Label inclusion threshold | Feature aggregation threshold | precision | recall | f1-score | time (s) |
| --- | --- | --- | --- | --- | --- |
| 5 | 1 | 0.72 | 0.73 | 0.72 | 665 | <<<<
| 5 | 10 | 0.66 | 0.70 | 0.67 | 411 |
| 5 | 50 | 0.67 | 0.71 | 0.68 | 477 |
| 5 | 100 | 0.67 | 0.71 | 0.68 | 572 |
| 5 | 500 | 0.65 | 0.70 | 0.66 | 406 |
| 2 | 100 | 0.68 | 0.71 | 0.68 | 924 |
| 3 | 100 | 0.68 | 0.72 | 0.69 | 658 |
| 5 | 100 | 0.67 | 0.71 | 0.68 | 572 | default
| 10 | 100 | 0.66 | 0.71 | 0.68 | 472 |
| 50 | 100 | 0.58 | 0.68 | 0.62 | 114 |
| 100 | 100 | 0.53 | 0.66 | 0.58 | 108 |

As expected, increasing the threshold for aggregating rare feature improves the training speed, without significantly impacting model performance. This suggests that rare features do not have strong predictive power. We can safely use a threshold value of 100 without sacrificing performance.?????????

Reducing the inclusion threshold for label frequency improves model performance up to a certain point, after which there are diminishing returns. We choose a threshold value of 5 attacks/group, which approaches the maximum 

## Cross-validation

We perform k-fold cross-validation to assess the ability of the model to generalize to unseen test cases. We must assume the attacks are independent and identically distributed, which may not be completely true as successful attacks may launch a chain of "copycat" attacks.

Given that some groups have significantly more attacks than others, we need to consider using stratified k-fold (instead of relying on randomization) to preserve relative class frequencies in each fold.

In [ ]:
class SuppressPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = None
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout
        
def cross_validate(group_thres=5, cate_thres=100, k=4):
    with SuppressPrints():
        df = import_data(variable_list = 
                         '/Users/richard/Documents/Fellowship.ai/variables LONGER.txt')
        df.reset_index(drop=True, inplace=True)
        kfolds = KFold(n_splits=k, shuffle=True)
        splits = kfolds.split(df)
        metrics = []
        for train_index, test_index in splits:
            metric = classify(group_thres, cate_thres, train_index, test_index, df)
            metrics.append(metric)
        
        scores = pd.DataFrame(columns=['precision','recall','fscore'])
        scores.loc['average'] = [np.mean(m) for m in list(zip(*metrics))[:-1]]
        scores.loc['std'] = [np.std(m) for m in list(zip(*metrics))[:-1]]
    print(scores)
    return(scores)

cross_validate()

The low standard deviation in our evaluation metrics over 4 k-folds suggests that our model is not overfitted to one particular training set. 

Since we have many groups with very few attacks, we use a weighted average of evaluation metrics (instead of micro/macro) to provide the most representative picture of model performance.